In [95]:
import sys 
import os
# sys.path = []
sys.path.insert(1,os.path.abspath("../src/"))
sys.path.insert(1,os.path.abspath("../src/finn/"))
sys.path.insert(1,os.path.abspath("../src/qonnx/"))
print(sys.path)
clean_sys_path = sys.path

def print_name(key):
    print("--"*20,key, "--"*20)
    

['/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/juypter_notebooks', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src/qonnx', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src/finn', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src/qonnx', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src/finn', '/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/src', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/share/pynq-venv/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/local/lib/python3.10/dist-packages/onnx-1.12.0-py3.10-linux-aarch64.egg', '/usr/local/lib/python3.10/dist-packages/typing_extensions-4.4.0-py3.10.egg', '/usr/local/lib/python3.10/dist-packages/qonnx-0.0.0-py3.10.egg', '/usr/local/lib/python3.10/dist-packages/toposort-1.7-py3.10.egg', '/usr/local/lib/python3.10/dist-packages/

# Create dummy inputs and store in the correct model directory 
For the moment want to create dummy input data, and keep this constant such that it does not change. Use this data to verify that the hw + cpu run has the same output as the expected output.


In [97]:
import numpy as np
import os
import shutil

clear_data = False

# Create a directory with relative info
model_dict = {}

# Get the onnx model directory
md = os.path.abspath("../models")
op_type = "Conv"

# Get the model hw directory where the driver file is located.
build_dir = os.path.abspath("../build-KV260/")

# Get split names for created hw files
split_dirs = [s for s in os.listdir(os.path.join(build_dir,op_type))]

# Fill the model dict
for split_name in split_dirs:
    print_name(split_name)
    model_base_folder = os.path.join(md,op_type,split_name)
    driver_dir = os.path.join(model_base_folder,"driver")
    bitfile_dir = os.path.join(model_base_folder,"bitfile")
    data_dir = os.path.join(model_base_folder,"data")
    if clear_data:
        try:
            shutil.rmtree(data_dir)
            shutil.rmtree(bitfile_dir)
            shutil.rmtree(driver_dir)
        except OSError as e:
            pass        
    else:
        shutil.copytree(os.path.join(build_dir,op_type,split_name,"deploy"),
                        model_base_folder, 
                        dirs_exist_ok=True
                       )
        
    ! ls {model_base_folder}
    model_dict[split_name.replace('_out0','')] = {"model_dir": model_base_folder, 
                                                  "driver_dir": driver_dir, 
                                                  "bitfile_dir":bitfile_dir,
                                                  "data_dir": data_dir}

---------------------------------------- Conv_6_out0 ----------------------------------------
bitfile  driver		   partition_0_ort.onnx  partition_1_ort.onnx
data	 partition_0.onnx  partition_1.onnx
---------------------------------------- Conv_10_out0 ----------------------------------------
bitfile  driver		   partition_0_ort.onnx  partition_1_ort.onnx
data	 partition_0.onnx  partition_1.onnx
---------------------------------------- Conv_0_out0 ----------------------------------------
bitfile  driver		   partition_0_ort.onnx  partition_1_ort.onnx
data	 partition_0.onnx  partition_1.onnx
---------------------------------------- Conv_2_out0 ----------------------------------------
bitfile  driver		   partition_0_ort.onnx  partition_1_ort.onnx
data	 partition_0.onnx  partition_1.onnx
---------------------------------------- Conv_4_out0 ----------------------------------------
bitfile  driver		   partition_0_ort.onnx  partition_1_ort.onnx
data	 partition_0.onnx  partition_1.onnx
---------

##### Eventually want to supply a image and see the output

In [71]:
# Create dummy data for the moment
for k,v in model_dict.items():
    sd = v["data_dir"]
    if not os.path.exists(sd):
        os.mkdir(sd)
        x = np.ones([1,3,224,224])
        np.save(f"{sd}/{k}_input.npy",x)
        v["data_dir"] = sd
    ! ls {v["data_dir"]}

Conv_6_input.npy
Conv_10_input.npy
Conv_0_input.npy
Conv_2_input.npy
Conv_4_input.npy
Conv_8_input.npy


# Get golden output 
For each of the inputs get the expected output from the baseline model

In [75]:
from qonnx.core.modelwrapper import ModelWrapper
from onnxruntime_extensions import get_library_path
import onnxruntime as ort
from custom_ort_functions import set_multithreshold_default

# Load input streamlined model
mf = os.path.abspath("../models/mobilenet_streamline.onnx")
model = ModelWrapper(mf)
model = model.transform(DoubleToSingleFloat())
set_multithreshold_default(model,mf.replace('.onnx',"_ort.onnx"))

# Set up a session for 
so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
sess = ort.InferenceSession(mf.replace('.onnx',"_ort.onnx"), so)
print(sess.get_inputs()[0].shape)

for k,v in model_dict.items():
    data_dir = v["data_dir"]
    x = np.load(f"{data_dir}/{k}_input.npy")
    inp_dict = {sess.get_inputs()[0].name: x.astype(np.float32)}
    _gld_res = sess.run([],inp_dict)
    print(_gld_res)
    np.save(f"{data_dir}/{k}_gld_output.npy",_gld_res)
#     ! ls {data_dir}

[1, 3, 224, 224]
Conv_6_gld_output.npy  Conv_6_input.npy
Conv_10_gld_output.npy	Conv_10_input.npy
Conv_0_gld_output.npy  Conv_0_input.npy
Conv_2_gld_output.npy  Conv_2_input.npy
Conv_4_gld_output.npy  Conv_4_input.npy
Conv_8_gld_output.npy  Conv_8_input.npy


# Perform the hardware run for the dummy data

In [98]:
for k,v in model_dict.items():
    print_name(k)
    x = os.path.join(v['data_dir'],f"{k}_input.npy")
    y = os.path.join(v['data_dir'],f"{k}_hw_output.npy")
#     ! ls {v["bitfile_dir"]}
    x_hw = np.load(x).reshape((1, 224, 224, 3))
    x_hw_f =os.path.join(v['data_dir'],f"{k}_hw_input.npy") 
    np.save(x_hw_f,x_hw)
    if not os.path.isfile(y):
        ! python {v["driver_dir"]}/driver.py --inputfile={x_hw_f} --outputfile={y} --bitfile={v["bitfile_dir"]}/finn-accel.bit
    y_hw = np.load(y)
    print(y_hw.shape)

---------------------------------------- Conv_6 ----------------------------------------
(1, 56, 56, 128)
---------------------------------------- Conv_10 ----------------------------------------
(1, 28, 28, 256)
---------------------------------------- Conv_0 ----------------------------------------
(1, 111, 111, 32)
---------------------------------------- Conv_2 ----------------------------------------
(1, 111, 111, 64)
---------------------------------------- Conv_4 ----------------------------------------
(1, 56, 56, 128)
---------------------------------------- Conv_8 ----------------------------------------
(1, 28, 28, 256)


In [5]:
from qonnx.core.datatype import DataType
from driver_base import FINNExampleOverlay

io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : [DataType['UINT8']],
    "odt" : [DataType['INT24']],
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : [(1, 224, 224, 3)],
    "oshape_normal" : [(1, 111, 111, 32)],
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : [(1, 224, 224, 3, 1)],
    "oshape_folded" : [(1, 111, 111, 2, 16)],
    "ishape_packed" : [(1, 224, 224, 3, 1)],
    "oshape_packed" : [(1, 111, 111, 2, 48)],
    "input_dma_name" : ['idma0'],
    "output_dma_name" : ['odma0'],
    "number_of_external_weights": 0,
    "num_inputs" : 1,
    "num_outputs" : 1,
}


def exe_mode(batch_size = 1,
    bitfile = "../bitfile/finn-accel.bit",
    inputfile = "input.npy",
    outputfile = "output.npy",
    runtime_weight_dir = "runtime_weights/",
    io_shape_dict = None
            ):
        platform = "zynq-iodma"
        accel = FINNExampleOverlay(
        bitfile_name = bitfile, platform = platform,
        io_shape_dict = io_shape_dict, batch_size = batch_size,
        runtime_weight_dir = runtime_weight_dir
        )
        print(io_shape_dict)
     # load desired input .npy file(s)
        ibuf_normal = []
        for ifn in [inputfile]:
            ibuf_normal.append(np.load(ifn))
        obuf_normal = accel.execute(ibuf_normal)
        if not isinstance(obuf_normal, list):
            obuf_normal = [obuf_normal]
        for o, obuf in enumerate(obuf_normal):
            np.save(outputfile, obuf)

In [17]:
from importlib import reload
exe = False
past_driver = ''
for k,v in model_dict.items():
    print("-"*20,k,"-"*20)
    
    hw_deploy_dir = v["hw_deploy_dir"]  
    driver_dir = os.path.abspath(os.path.join(hw_deploy_dir,"../driver"))
#     print(driver_dir)
    sys.path.append(driver_dir)
    import driver    
#     io_shape_dict = PACKAGE.io_shape_dict
    if driver.__file__ != past_driver:
        import driver    
        past_driver = driver.__file__
    print(driver.__file__)

    io_shape_dict = driver.io_shape_dict
        
#     print(io_shape_dict.__file__)
    print(io_shape_dict['oshape_normal'])

    
    data_dir = v["data_dir"]
    # Load and reshape the input
    x = np.load(f"{data_dir}/{k}_input.npy")
    x = x.reshape(io_shape_dict["ishape_normal"][0])
    x = x.astype(np.uint8)
    # Save the reshaped data for hw input
    np.save(f"{data_dir}/{k}_hw_input.npy",x)
    # execute hw run 
#     print(f"{data_dir}/{k}_hw_output.npy")
#     if os.path.exists(f"{data_dir}"):
#         for f in os.listdir(f"{data_dir}"):
#             if f"{k}_hw_output.npy" == f:
#                 exe = False
    if exe:
        exe_mode(bitfile=f"{hw_deploy_dir}/bitfile/finn-accel.bit",
                 inputfile=f"{data_dir}/{k}_hw_input.npy",
                 outputfile=f"{data_dir}/{k}_hw_output.npy")
    sys.path.remove(driver_dir)
# print(sys.path)
# sys.path = clean_sys_path


-------------------- Conv_6 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.py
[(1, 56, 56, 128)]
-------------------- Conv_10 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.py
[(1, 56, 56, 128)]
-------------------- Conv_0 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.py
[(1, 56, 56, 128)]
-------------------- Conv_2 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.py
[(1, 56, 56, 128)]
-------------------- Conv_4 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.py
[(1, 56, 56, 128)]
-------------------- Conv_8 --------------------
/home/root/jupyter_notebooks/MSC_Git_files/mobilenet_v1/build-KV260/Conv/Conv_6_out0/driver/driver.p

In [101]:
for k,v in model_dict.items():
#     Get hw output
    y_hw = np.load(v['data_dir'] + f'/{k}_hw_output.npy')
#     Create a ort session
    mf = v['model_dir']+'/partition_1.onnx'
    mf_update = mf.replace('.onnx','_ort.onnx')

    set_multithreshold_default(ModelWrapper(mf),mf_update)

    so = ort.SessionOptions()
    
    so.register_custom_ops_library(get_library_path())
    sess = ort.InferenceSession(mf_update, so)
    print(y_hw.shape,sess.get_inputs()[0].shape)

(1, 111, 111, 32) [1, 128, 56, 56]
(1, 111, 111, 32) [1, 256, 28, 28]
(1, 111, 111, 32) [1, 32, 111, 111]
(1, 111, 111, 32) [1, 64, 111, 111]
(1, 111, 111, 32) [1, 128, 56, 56]
(1, 111, 111, 32) [1, 256, 28, 28]
